In [1]:
from pyspark.ml.regression import LinearRegression,LinearRegressionModel
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql import Row
from pyspark.sql.functions import udf
from pyspark.ml.param import Param, Params
from pyspark.sql.functions import *
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
import numpy as np
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, IntegerType
from pyspark import SparkContext
from pyspark.sql import SparkSession
import pickle

sc = SparkContext("local",'app')
spark = SparkSession.builder.appName('name').config('spark.sql.shuffle.partitions',10).getOrCreate()

ImportError: No module named pyspark.ml.regression

## 1. 
Add one line to find the most frequent word. Output this word and its frequency.

Hint: Use sortBy(), reduce(), or max()

In [3]:
from operator import add
lines = sc.textFile('../data/README.md')
counts = lines.flatMap(lambda x: x.split()) \
              .map(lambda x: (x, 1)) \
              .reduceByKey(add)

In [6]:
# answer
sort_value = counts.sortBy(lambda x:x[1],ascending=False)
print sort_value.take(1) #print out the most frequent word

[(u'the', 582)]


## 2.
Modify the word count example above, so that we only count the frequencies of those words consisting of 5 or more characters.

In [10]:
#answer
counts_filted = counts.filter(lambda count:len(count[0])>=5)
print counts_filted.take(6)

[(u'limited', 2), (u'represent', 2), (u'prefix', 1), (u'Until', 1), (u'(approximate)', 1), (u'Metadata', 1)]


## 3.
Consider the following piece of code:

What's its output?

In [14]:
A = sc.parallelize(xrange(1, 100))
t = 50
B = A.filter(lambda x: x < t)
print B.count()
t = 10
C = B.filter(lambda x: x > t)  # it would generate the C and B from A once again, at this moment, t =10
print C.count()

49
0


## 4
The intent of the code above is to get all numbers below 50 from A and put them into B, and then get all numbers above 10 from B and put them into C.  Fix the code so that it produces the desired behavior, by adding one line of code.  You are not allowed to change the existing code.

In [15]:
A = sc.parallelize(xrange(1, 100))
t = 50
B = A.filter(lambda x: x < t)
B.cache()  # answer lines,. B was persisted in disk.
t = 10
C = B.filter(lambda x: x > t)
print C.count()

39


## 5.
Modify the PMI example by sending a_dict and n_dict inside the closure. Do not use broadcast variables.

In [23]:
lines = sc.textFile('../data/adj_noun_pairs.txt')

pairs = lines.map(lambda l: tuple(l.split())).filter(lambda p: len(p)==2)
pairs.cache()

N=pairs.count()

pair_freqs = pairs.map(lambda p: (p,1)).reduceByKey(lambda f1, f2: f1 + f2) \
                  .filter(lambda pf: pf[1] >= 1)

# Computing the frequencies of the adjectives and the nouns
a_freqs = pairs.map(lambda p: (p[0],1)).reduceByKey(lambda x,y: x+y)
n_freqs = pairs.map(lambda p: (p[1],1)).reduceByKey(lambda x,y: x+y)

In [ ]:
# answer
n_dict = n_freqs.collectAsMap()  #change
a_dict =a_freqs.collectAsMap()  #change

In [30]:
from math import *
# Computing the PMI for a pair.
# Computing the PMI for a pair.
def pmi_score(pair_freq):
    global a_dict,n_dict
    w1, w2 = pair_freq[0]
    f = pair_freq[1]
    pmi = log(float(f)*N/(a_dict[w1]*n_dict[w2]), 2)  #change
    return pmi, (w1, w2)

# Computing the PMI for all pairs.
scored_pairs = pair_freqs.map(pmi_score)
# Printing the most strongly associated pairs. 
scored_pairs.top(10)

[(21.59271342368657, (u'\u9ad8\u91ce\u8c46\u8150', u'k\u014dyad\u014dfu')),
 (21.59271342368657, (u'\u997f', u'\xe8')),
 (21.59271342368657, (u'\u9648\u8bda', u'\u9673\u8aa0')),
 (21.59271342368657, (u'\u9648\u6bc5', u'\u9673\u6bc5')),
 (21.59271342368657, (u'\u8a00\u8449engrish', u'\u3002')),
 (21.59271342368657, (u'\u7d2f', u'l\xe8i')),
 (21.59271342368657, (u'\u7b28', u'b\xe8n')),
 (21.59271342368657, (u'\u6ce2\u52d5\u62f3', u'Hadouken')),
 (21.59271342368657, (u'\u6cc9\u6f33\u90fd\u6307\u63ee\u4f7f', u'xiao4')),
 (21.59271342368657, (u'\u5f8c\u9ad8\u5009\u9662', u'Go-Takakura-in'))]

In [ ]:
pair_freqs = pairs.map(lambda p: (p,1)).reduceByKey(lambda f1, f2: f1 + f2)
print pair_freqs.sortBy(lambda x:x[1],ascending=False).map(lambda x:x[0]).take(10)